In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generative AI Knowledge Base model predictions

To run this notebook, make sure you have uploaded at least one document into your knowledge base.

> ⭐️ If you haven't, follow the [**Uploading documents and query model** tutorial](https://console.cloud.google.com/products/solutions/deployments?walkthrough_id=panels--sic--generative-ai-knowledge-base_toc).

Before you begin, make sure all the dependencies are installed.

In [ ]:
!pip install google-cloud-aiplatform google-cloud-firestore

## Overview

A **Large Language Model (LLM)** can be very good at answering general questions.
But it might not do as well to answer questions from your documents on its own.

The LLM will answer only from what it learned from its _training dataset_.
Your documents might include information or words that weren't on that dataset.
Or they might be used in a different or more specialized context.

This is where **Vector Search** comes into place.
Each time you upload a document, the Cloud Function webhook processes it.
When a document is processed, each individual page is _indexed_.
This allows us to not only find documents, but the specific pages.

The relevant pages can then be used as _context_ for the LLM to answer the question.
This _grounds_ the model to answer questions based on the documents only.
Without this, the model might give wrong answers, or _hallucinations_.

## My Google Cloud resources

Fill in your project ID, the
[Google Cloud location](https://cloud.google.com/about/locations)
you want to use, and your
Vector Search index endpoint ID.
If you followed the tutorial, the deployed index ID should be `deployed_index`, otherwise change it to the ID you chose.

You can find your Vector Search index endpoint ID in the [Index endpoints tab](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints).

> 💡 The Vector Search index endpoint ID looks like a number, like `1234567890123456789`.

Run the following cell to set up your resources and authenticate to your account.

In [1]:
# @title
from google.colab import auth

project_id = "" # @param {type:"string"}
location = "us-central1" # @param {type:"string"}
index_endpoint_id = "" # @param {type:"string"}
deployed_index_id = "deployed_index" # @param {type:"string"}

auth.authenticate_user(project_id=project_id)

The first step is to initialize the Vertex AI client library using the location of your choice.

In [11]:
import vertexai
from google.cloud import aiplatform

vertexai.init(location=location)
aiplatform.init(location=location)

## Get text embeddings

You can use the Gecko model to get embeddings from text.
For more information, see the
[Get text embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings)
page.

In [3]:
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

def get_text_embedding(text: str) -> list[float]:
    task = 'RETRIEVAL_DOCUMENT'
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko")
    return model.get_embeddings([TextEmbeddingInput(text, task)])[0].values


# Convert the question into an embedding.
question = "What are LFs and why are they useful?"
question_embedding = get_text_embedding(question)
print(f"Embedding dimensions: {len(question_embedding)}")

Embedding dimensions: 768


## Find document context

All the documents you have processed have been indexed into your Vector Search index.
You can query for the closest embeddings to a given embedding from your Vector Search index endpoint.

> 💡 If you haven't processed any documents yet, you won't get any results.

In [5]:
from itertools import groupby

def find_document(question: str, index_endpoint_id: str, deployed_index_id: str) -> tuple[str, int]:
    # Get embeddings for the question.
    embedding = get_text_embedding(question)

    # Find the closest point from the Vector Search index endpoint.
    endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_id)
    point = endpoint.find_neighbors(
        deployed_index_id=deployed_index_id,
        queries=[embedding],
        num_neighbors=1,
    )[0][0]

    # Get the document name and page number from the point ID.
    (filename, page_number) = point.id.split(':', 1)
    return (filename, int(page_number))

# Query the Vector Search index for the most relevant page.
(filename, page_number) = find_document(question, index_endpoint_id, deployed_index_id)
print(f"{filename=} {page_number=}")

filename='9410009v1.pdf' page_number=3


## Get document text

When documents were processed, their text was stored in Firestore as well.
The Vector Search query returned the relevant documents with their page numbers.
With this you can download the document's pages and give only the most relevant page to the model.

In [6]:
from google.cloud import firestore

def get_document_text(filename: str, page_number: int) -> str:
    db = firestore.Client(database='knowledge-base-database')
    doc = db.collection("documents").document(filename)
    return doc.get().get('pages')[page_number]

# Download the document's page text from Firestore.
context = get_document_text(filename, page_number)
print(f"{context[:1000]}\n...\n...")

EN SEM IND
FR SEM IND
VAR
REST {Magn( 1 )}
VAR
REST {Magn(
The interlingual status of the lexical function is
self-evident. Any occurrence of Magn will be left
intact during transfer and it will be the generation
component that ultimately assigns a monolingual
lexical entry to the LF.6
3.2 Problems
Lexical Functions abstract away from certain nu-
ances in meaning and from different syntactic re-
alizations. We discuss some of the problems raised
by this abstraction in this section.
Overgenerality An important problem stems
from the interpretation of LFs implied by their
use as an interlingua namely that the mean-
ing of the collocate in some ways reduces to the
meaning implied by the lexical function. This in-
terpretation is trouble-free if we assume that LFs
always deliver unique values; unfortunately cases
to the contrary can be readily observed. An exam-
ple attested from our corpus was the range of ad-
verbial constructions possible with the verbal head
oppose: adamantly, bitterly

## Ask a foundational model

With the relevant context ready, you can now make a _prompt_ that includes both the context and the question.

Here's Gemini's response.
Note that Gemini responds in [Markdown](https://www.markdownguide.org).

In [90]:
from vertexai.generative_models import GenerativeModel

# Ask the foundational model.
model = GenerativeModel(
    model_name="gemini-1.5-flash-002",
    system_instruction=context,
)
answer = model.generate_content(question).text

print("QUESTION:")
print(question)
print()
print("ANSWER:")
print(answer)

QUESTION
What are LFs and why are they useful?

ANSWER:
## What are Lexical Functions (LFs)?

LFs are a key tool in computational linguistics and machine translation, used to represent the potential relationships between words within a language. They're essentially a type of interlingual annotation, meaning they offer a language-neutral way to describe these relationships, independent of the specific words used in any particular language.

## How LFs are used:

- **Collocation Analysis:** LFs are particularly powerful when analyzing collocations, which are frequent co-occurrences of words. By capturing the relationships between words in collocations, LFs help us understand the meaning and nuances of these combinations, even across different languages.
- **Translation Support:** This deeper understanding of collocations provided by LFs is invaluable for machine translation. LFs help ensure that the meaning of the original text is accurately reflected in the translated text, taking into 

## (Optional) Ask your tuned model

If you want to tune a model, follow the [**Fine-tune an LLM model** tutorial](https://console.cloud.google.com/products/solutions/deployments?walkthrough_id=panels--sic--generative-ai-knowledge-base_toc).

First, find the tuning job ID for your tuned model.

In [ ]:
from vertexai.preview.tuning import sft

for tuning_job in sft.SupervisedTuningJob.list():
    model_name = tuning_job.gca_resource.tuned_model_display_name
    tuning_job_id = tuning_job.resource_name
    print(f"{model_name}: {tuning_job_id}")

Copy your tuning job ID and paste it below.
Don't forget to run the cell to define the `tuning_job_id` variable.

In [80]:
tuning_job_id = "" # @param {type:"string"}

In [93]:
from vertexai.generative_models import GenerativeModel
from vertexai.preview import tuning
from vertexai.preview.tuning import sft

tuning_job = sft.SupervisedTuningJob(tuning_job_id)
assert tuning_job.tuned_model_endpoint_name, "Please wait until the tuning job finishes."

tuned_model = GenerativeModel(
    model_name=tuning_job.tuned_model_endpoint_name,
    system_instruction=context,
)
answer = tuned_model.generate_content(question).text

print("QUESTION:")
print(question)
print()
print("ANSWER:")
print(answer)

QUESTION:
What are LFs and why are they useful?

ANSWER:
Lexical functions (LFs) are functions that operate on lexemes. They are useful because they can be used to generate synonyms.
